In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [2]:
from google.adk.tools import google_search, AgentTool, FunctionTool
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.agents import Agent, SequentialAgent, LoopAgent, ParallelAgent
from google.genai import types

In [3]:
retry_configs = types.HttpRetryOptions(
    attempts=5,
    initial_delay=1,
    exp_base=7,
    http_status_codes=[429,500,503,504]
)

<img width="800" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/multi-agent-team.png" alt="Multi-agent Team" />

In [4]:
research_agent = Agent(
    name="research_agent",
    model=Gemini(model='gemini-2.5-flash', retry_options=retry_configs),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings"
)

In [5]:
summarizer_agent = Agent(
    name="summarizer_agent",
    model=Gemini(model='gemini-2.5-flash', retry_options=retry_configs),
    instruction="""Read and profind research findings {research_findings}
    Create a concise summary as a bulleted list with 3-5 key points."""   ,
    output_key="final_summary"
)

In [6]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="research_coordinator",
    model=Gemini(model='gemini-2.5-flash', retry_options=retry_configs),
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
    1. First, you MUST call the `research_agent` tool to find relevant information on the topic provided by the user.
    2. Next, after receiving the research findings, you MUST call the `summarizer_agent` tool to create a concise summary.
    3. Finally, present the final summary clearly to the user as your response.""",
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)]
)

In [7]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("list all features in latest ubuntu update.")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/home/poojan/programs/personal-programs/5dgai/.venv/lib/python3.11/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > list all features in latest ubuntu update.


App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/home/poojan/programs/personal-programs/5dgai/.venv/lib/python3.11/site-packages/google/adk/agents", which implies app name "agents".
App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/home/poojan/programs/personal-programs/5dgai/.venv/lib/python3.11/site-packages/google/adk/agents", which implies app name "agents".


research_coordinator > Here's a concise summary of the latest Ubuntu updates:

*   **Ubuntu 25.10 "Questing Quokka"** (Oct 2025) fully transitions to Wayland, enhances security with Rust-based `sudo` and experimental TPM encryption, and improves NVIDIA driver stability.
*   **Ubuntu 24.04 LTS "Noble Numbat"** (Apr 2024) offers extended support (up to 12 years with Pro), features a redesigned Flutter-based installer and App Center, and adopts Pipewire as the default sound server.
*   Both releases include updated Linux kernels and GNOME desktop environments, bringing UI refinements, performance enhancements, and improved hardware support.
*   Ubuntu 24.04 LTS further introduces advanced window tiling, updated developer toolchains, and a dedicated firmware updater.


##### Let's do it in better way

In [22]:
# Outline Agent: Creates the initial blog post outline.
outline_agent = Agent(
    name="OutlineAgnet",
    model=Gemini(model = 'gemini-2.5-flash', retry_options=retry_configs),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought
    5. Always use google search tool for latest information""",
    output_key="blog_outline",  # The result of this agent will be stored in the session state with this key.
    tools=[google_search]
)

In [ ]:
writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(model = 'gemini-2.5-flash', retry_options=retry_configs),
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.
    Use google search for latest information""",
    output_key="blog_draft",
    # tools=[google_search]
)

In [24]:
editor_agent = Agent(
    name="EditorAgent",
    model=Gemini(model = 'gemini-2.5-flash', retry_options=retry_configs),
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog"
)

In [25]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent]
)

In [27]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a blog post about the benefits of multi-agent systems for software developers")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/home/poojan/programs/personal-programs/5dgai/.venv/lib/python3.11/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Write a blog post about the benefits of multi-agent systems for software developers
OutlineAgnet > ## Unleash Collaborative Intelligence: Why Multi-Agent Systems Are Your Next Development Superpower

### Introduction Hook
Modern software development is increasingly complex, demanding solutions that can handle intricate problems, dynamic environments, and rapid changes. While single AI models have made significant strides, a new paradigm is emerging: Multi-Agent Systems (MAS). These systems, composed of multiple autonomous AI agents collaborating to achieve shared goals, are revolutionizing how software is built, tested, and delivered. In fact, Gartner predicts that by 2028, AI agents will autonomously handle at least 15% of workplace decisions, ranking AI agents first on Gartner's list of top Strategic Technology Trends for 2025. For software developers, understanding and leveraging MAS is no longer optional—it's a competitive edge.



#### Parallel Workflows - Independent Researchers

<img width="600" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/parallel-agent.png" alt="Parallel Agent" />

In [30]:
# Tech Researcher: Focuses on AI and ML trends.
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_configs
    ),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [31]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_configs
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [32]:
# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_configs
    ),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",  # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [34]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_configs
    ),
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary"
)


In [35]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher]
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent]
)

In [36]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily executive briefing on Tech, Health, and Finance"
)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/home/poojan/programs/personal-programs/5dgai/.venv/lib/python3.11/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
HealthResearcher > Here's a concise executive briefing on recent breakthroughs in Tech, Health, and Finance:

**Health:** Gene therapy is a rapidly advancing frontier, with recent successes in treating inherited hearing loss and sickle cell anemia. Within 5 years, expect wider applications for genetic diseases. CAR T-cell therapy shows promise for brain cancers, potentially impacting treatment within 3-7 years. Additionally, iPS cell-derived corneal transplants are restoring vision, with larger trials planned, suggesting broader eye disease applications in 5-10 years.

**Tech:** Agentic AI, capable of autonomous workflow execution, is a major development with business impacts expected within 1-3 years. Advancements in quantum computing are progressing towards practical applications in material science and drug development, with potential impacts in 5-10 years.

**Finance:**

#### Loop Workflows - The Refinement Cycle

<img width="250" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/loop-agent.png" alt="Loop Agent" />

In [37]:
# This agent runs ONCE at the beginning to create the first draft.
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_configs
    ),
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story",  # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


In [38]:
# This agent's only job is to provide feedback or the approval signal. It has no tools.
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_configs
    ),
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",  # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


In [39]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

In [41]:
# This agent refines the story based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_configs
    ),
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story",  # It overwrites the story with the new, refined version.
    tools=[
        FunctionTool(exit_loop)
    ],  # The tool is now correctly initialized with the function reference.
)

In [ ]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent,                                                                                                                                                                      ]
)

In [43]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a short story about a lighthouse keeper who discovers a mysterious, glowing map")

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/home/poojan/programs/personal-programs/5dgai/.venv/lib/python3.11/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map
InitialWriterAgent > Elias had known the lighthouse's granite bones intimately for thirty years. Each creak of the lantern room, each groan of the wind against the glass, was a familiar lullaby. One blustery Tuesday, while dusting the seldom-used archives in the basement, his fingers brushed against a loose stone. Behind it lay a rolled parchment, tied with faded crimson ribbon. Unfurling it, Elias gasped. It was a map, but unlike any he'd ever seen. The ink wasn't ink at all, but a shimmering, bioluminescent substance that pulsed with a soft, ethereal light. Strange symbols, reminiscent of ancient constellations, marked out a coastline that looked eerily like his own, yet subtly alien. A single, brilliant star pulsed at its center, a beacon on the paper itself. Elias, accustomed to the predictable rhythm of the sea, felt a tremor of something entire

#### Choosing the Right Pattern
#### Decision Tree: Which Workflow Pattern?

<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/agent-decision-tree.png" alt="Agent Decision Tree" />

### Quick Reference Table

| Pattern | When to Use | Example | Key Feature |
|---------|-------------|---------|-------------|
| **LLM-based (sub_agents)** | Dynamic orchestration needed | Research + Summarize | LLM decides what to call |
| **Sequential** | Order matters, linear pipeline | Outline → Write → Edit | Deterministic order |
| **Parallel** | Independent tasks, speed matters | Multi-topic research | Concurrent execution |
| **Loop** | Iterative improvement needed | Writer + Critic refinement | Repeated cycles |

SyntaxError: invalid syntax (692728953.py, line 1)